In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import inspect

from skimage import transform as tf
from skimage import morphology as mph

from scipy.stats import mode
from scipy.ndimage import binary_dilation

import peakutils

from defdap.file_readers import DICDataLoader
from defdap import base
from defdap.quat import Quat

from defdap import defaults
from defdap.plotting import MapPlot, GrainPlot
#from defdap.inspector import GrainInspector
from defdap.utils import reportProgress

%matplotlib qt5
from defdap import quat
from defdap import ebsd
from defdap import hrdic
from defdap import plotting

In [5]:
#defdap.plotting.MapPlot    
def create(  
    cls, callingMap, mapData,
    fig=None, figParams={}, ax=None, axParams={},
    plot=None, makeInteractive=False,
    plotColourBar=False, vmin=None, vmax=None, cmap=None, clabel="",
    plotGBs=False, dilateBoundaries=False, boundaryColour=None,
    plotScaleBar=False, scale=None,
    highlightGrains=None, highlightColours=None, highlightAlpha=None,
    **kwargs
    ):
    
    if plot is None:
        plot = cls(callingMap, fig=fig, ax=ax, axParams=axParams,
                    makeInteractive=makeInteractive, **figParams)
    if mapData is not None:
        plot.addMap(mapData, cmap=cmap, vmin=vmin, vmax=vmax, **kwargs)
    if plotColourBar:
        plot.addColourBar(clabel)
    if plotGBs:
        plot.addGrainBoundaries(
            colour=boundaryColour, dilate=dilateBoundaries, kind=plotGBs
        )
    if highlightGrains is not None:
        plot.addGrainHighlights(
            highlightGrains,
            grainColours=highlightColours, alpha=highlightAlpha
        )
    if plotScaleBar:
        plot.addScaleBar(scale=scale)
    return plot

#defdap.optical.Map
class Map(base.Map):
    def __init__(self, path, fname, extension='png'):
        """Initialise class and import DIC data from file.

        Parameters
        ----------
        path : str
            Path to file.
        fname : str
            Name of file without extension.
        extension : str
            File extension (default is 'png').
        """
        
        self.extension = extension
        self.path = path                    # file path
        self.fname = fname                  # file name
    
        # Call base class constructor
        super(Map, self).__init__()

        # Initialise variables
        self.format = None      # Software name
        self.version = None     # Software version
        self.binning = None     # Sub-window size in pixels
        self.xdim = None        # Size of map along x (from header)
        self.ydim = None        # Size of map along y (from header)

        self.xc = None          # x coordinates
        self.yc = None          # y coordinates
        self.xd = None          # x displacement
        self.yd = None          # y displacement
        
        self.corrVal = None     # Correlation value
        self.ebsdMap = None     # EBSD map linked to DIC map
        self.ebsdTransform = None  # Transform from EBSD to DIC coordinates
        self.ebsdTransformInv = None  # Transform from DIC to EBSD coordinates
        self.ebsdGrainIds = None
        self.patternImPath = None  # Path to BSE image of map
        self.highlightAlpha = 0.6
        self.bseScale = None  # Size of a pixel in the correlated images
        self.patScale = None  # Size of pixel in loaded pattern relative to pixel size of DIC data

        # Try to load the optical image and handle errors
        try:
            self.optical = plt.imread(self.path + f"{self.fname}.{self.extension}")
        except FileNotFoundError:
            print(f"Error: File {self.fname}.{self.extension} not found in path {self.path}")
            self.optical = None  # Set to None if the image is not found

In [6]:
optical = Map('./','F5-pol-Cy50')
mapData = np.array(optical)
mapData

array(<__main__.Map object at 0x000001CA300B2400>, dtype=object)

In [7]:
create(callingMap = optical,mapData=mapData)

TypeError: create() missing 1 required positional argument: 'cls'

In [ ]:
int('a')

In [ ]:
    def plot_map(self,plot_axis='off', makeInteractive=False,**kwargs):
        "Plots the optical image stored in the instance."
        if self.optical is not None:
            plt.imshow(self.optical)
            plt.axis(plot_axis)
            plt.title('Optical Image')
            plt.show()
            return ax
        else:
            print("No optical image to display.")
            
        def setHomogPoint(self, points=None, display='polarised', **kwargs):
        """Set homologous points. Uses interactive GUI if points is None.

        Parameters
        ----------
        points : list, optional
        Homologous points to set.
        display : string, optional
        Use max shear map if set to 'maxshear' or pattern if set to 'pattern'.

        """
        if points is not None:
            self.homogPoints = points

        if points is None:
            if display == 'polarised' or display == 'darkfield':
                self.plotHomog = self.plot_map  # This plots the optical image
                binSize = self.patScale
                print(plotHomog)
            else:
                self.plotHomog = self.plot_map  # This plots the optical image
                binSize = self.patScale
                print(self.plotHomog)
        # Now call the base class's setHomogPoint with the correct binSize
            super(type(self), self).setHomogPoint(binSize=binSize, points=points, **kwargs)

In [2]:
class Map(base.Map):
    def __init__(self, path, fname, extension='png'):
        """Initialise class and import DIC data from file.

        Parameters
        ----------
        path : str
            Path to file.
        fname : str
            Name of file without extension.
        extension : str
            File extension (default is 'png').
        """
        
        self.extension = extension
        self.path = path                    # file path
        self.fname = fname                  # file name
    
        # Call base class constructor
        super(Map, self).__init__()

        # Initialise variables
        self.format = None      # Software name
        self.version = None     # Software version
        self.binning = None     # Sub-window size in pixels
        self.xdim = None        # Size of map along x (from header)
        self.ydim = None        # Size of map along y (from header)

        self.xc = None          # x coordinates
        self.yc = None          # y coordinates
        self.xd = None          # x displacement
        self.yd = None          # y displacement
        
        self.corrVal = None     # Correlation value
        self.ebsdMap = None     # EBSD map linked to DIC map
        self.ebsdTransform = None  # Transform from EBSD to DIC coordinates
        self.ebsdTransformInv = None  # Transform from DIC to EBSD coordinates
        self.ebsdGrainIds = None
        self.patternImPath = None  # Path to BSE image of map
        self.highlightAlpha = 0.6
        self.bseScale = None  # Size of a pixel in the correlated images
        self.patScale = None  # Size of pixel in loaded pattern relative to pixel size of DIC data

        # Try to load the optical image and handle errors
        try:
            self.optical = plt.imread(self.path + f"{self.fname}.{self.extension}")
        except FileNotFoundError:
            print(f"Error: File {self.fname}.{self.extension} not found in path {self.path}")
            self.optical = None  # Set to None if the image is not found
    
##################################################################################################################################################    
    
    def plot_map(self, plot_axis='off',makeInteractive=False):
        """Plots the optical image stored in the instance."""
        if self.optical is not None:
            fig, ax = plt.subplots()
            ax.imshow(self.optical)
            ax.axis(plot_axis)
            ax.set_title('Optical Image')
            plt.show()
            return ax  # Return the axis object for further manipulation
        else:
            print("No optical image to display.")
            return None
            

########################################################################################################################################################
    def setHomogPoint(self, points=None, display='polarised', **kwargs):
        """Set homologous points. Uses interactive GUI if points is None.

        Parameters
        ----------
        points : list, optional
            Homologous points to set.
        display : string, optional
            Use max shear map if set to 'maxshear' or pattern if set to 'pattern'.
        """
        if points is not None:
            self.homogPoints = points

        if points is None:
            if display in ['polarised', 'darkfield']:
                self.plotHomog = self.plot_map  # This plots the optical image
                binSize = self.patScale
                print(1)  # Use self.plotHomog
            else:
                self.plotHomog = self.plot_map  # This plots the optical image
                binSize = self.patScale
                print(2)  # Use self.plotHomog

        # Now call the base class's setHomogPoint with the correct binSize
        super().setHomogPoint(binSize=binSize, points=points, **kwargs)


In [3]:
optical = Map('./','F5-pol-Cy50')
optical.plot_map(plot_axis='on')

<AxesSubplot:title={'center':'Optical Image'}>

In [4]:
EbsdMap = ebsd.Map('../EBSD_data/post_deformed/f-5-test-region')

Loaded EBSD data (dimensions: 852 x 734 pixels, step size: 3.0 um)


In [5]:
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 2) #degrees
EbsdMap.findGrains(minGrainSize = 100) #pixels
EbsdMap.calcGrainMisOri(calcAxis = False)
EbsdMap.calcAverageGrainSchmidFactors(loadVector=[1,0,0])
#EbsdMap.rotateData()

Finished building quaternion array (0:00:03) 
Finished finding grain boundaries (0:00:06) 
Finished finding grains (0:00:05) 
Finished calculating grain misorientations (0:00:10) 
Finished calculating grain average Schmid factors (0:00:00) 


In [6]:
EbsdMap.plotIPFMap([1,0,0], plotScaleBar=True)#,plotGBs= True)

In [9]:
EbsdMap.plotHomog=EbsdMap.plotBandContrastMap
EbsdMap.setHomogPoint()
optical.setHomogPoint(display='polarised')

1


AttributeError: 'AxesSubplot' object has no attribute 'addPoints'

In [11]:
plotting.MapPlot.create(base.Map,optical)

TypeError: Image data of dtype object cannot be converted to float